# Multivariate Gaussian Example

In [ ]:
import sys
sys.path.append("../")
import numpy as np
import matplotlib.pyplot as plt
import time

import multiprocessing as mp

from ebc.sequential.iterative_with_convexification import SensitivityBasedFW
from ebc.gaussian import gaussian_multivariate_log_likelihood, gaussian_KL

from splitting import split_based_on_ML, split_randomly, distribute
from parallelization import parallelize

from sklearn.mixture import GaussianMixture
import pickle

from analyze_distribution import plot_coresets

## Log-likelihoods Definitions

In [ ]:
# Define log-likelihood

def log_likelihood(params, X, y, weights):
    '''
    Returns:
    ----------
    log_lik: np.ndarray(shape = X.shape[0])
    '''
    d = X.shape[1]
    mu = params[:d].reshape(-1, 1)
    sigma = np.diag(params[d:].reshape(-1, 1)[:, 0])
    return np.diag(gaussian_multivariate_log_likelihood(X.T, mu, sigma)).reshape(-1, 1)

def summed_log_likelihood(params, X, y, weights):
    return log_likelihood(params, X, y, weights).sum()

def negative_summed_log_likelihood(params, X, y, weights):
    return -summed_log_likelihood(params, X, y, weights)

# https://stats.stackexchange.com/questions/90134/gradient-of-multivariate-gaussian-log-likelihood
def grad_log_likelihood(params, X, y, weights):
    '''
    Returns:
    ----------
    grad_log_lik: np.ndarray(shape = X.shape[1])
    '''
    d = X.shape[1]
    mu = params[:d].reshape(-1, 1)
    sigma = np.diag(params[d:].reshape(-1, 1)[:, 0])
    return (-np.linalg.inv(sigma) @ (X.T - mu)).reshape(-1, X.shape[1])

def log_posterior(params, X, y, weights):
    return weights.T @ log_likelihood(params, X, y, weights)

## Data Generation

In [ ]:
# Generate data

N = 5000
d = 5

np.random.seed(123)

# Generate mean from MVN
theta = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d))

# Generate data
x = np.random.multivariate_normal(mean = theta, cov = np.identity(d), size = N)

# Parameters
sigma_0 = np.identity(d)
sigma = np.identity(d)
mu_0 = np.zeros(d).reshape(-1, 1)

# Full Gaussian posterior
sigma_full = np.linalg.inv(np.linalg.inv(sigma_0) + N * np.linalg.inv(sigma))
mu_full =  sigma_full @ (np.linalg.inv(sigma_0) @ mu_0 + np.linalg.inv(sigma) @ np.sum(x, axis = 0).reshape(-1, 1))

## Testing

In [ ]:
coreset_sizes = np.arange(100, 310, 10)
len(coreset_sizes)

In [ ]:
# Sequential
fkl_sequential = []
bkl_sequential = []
time_sequential = []

na = {"log_likelihood": log_likelihood,
      "log_likelihood_start_value": np.ones(2 * d),
      "S": 100,
      "log_likelihood_gradient": grad_log_likelihood,
      "approx": "MCMC",
      "MCMC_subs_size": int(0.7 * len(x)),
      "log_posterior": log_posterior,
      "log_posterior_start_value": np.ones(2 * d)}

for k in range(20):
      np.random.seed(120 + k)
      fkl_sequential_k = []
      bkl_sequential_k = []
      time_sequential_k = []

      w_seq = []

      for i in coreset_sizes:
            print(i, k)
            start = time.time()
            sbfw = SensitivityBasedFW(x)
            w, I = sbfw.run(k = i, likelihood_gram_matrix = None, norm = "2", norm_attributes = na)
            time_sequential_k.append(time.time() - start)

            # Calculate posterior approximation
            sigma_hat = 1 / (1 + np.sum(w)) * np.identity(d)
            mu_hat = sigma_hat @ (mu_0 + x.T @ w)

            fkl_sequential_k.append(gaussian_KL(sigma_full, sigma_hat, mu_full, mu_hat))
            bkl_sequential_k.append(gaussian_KL(sigma_hat, sigma_full, mu_hat, mu_full))

            w_seq.append(w)

      fkl_sequential.append(fkl_sequential_k)
      bkl_sequential.append(bkl_sequential_k)
      time_sequential.append(time_sequential_k)

print(f"FKL: {fkl_sequential}")
print(f"BKL: {bkl_sequential}")
print(f"Time: {time_sequential}")

In [ ]:
fkl_sequential = np.array(fkl_sequential)
bkl_sequential = np.array(bkl_sequential)
time_sequential = np.array(time_sequential)

In [ ]:
# Parallel
fkl_parallel = []
bkl_parallel = []
time_parallel = []

na = {"log_likelihood": log_likelihood,
      "log_likelihood_start_value": np.ones(2 * d),
      "S": 100,
      "log_likelihood_gradient": grad_log_likelihood,
      "approx": "MCMC",
      "MCMC_subs_size": int(0.1 * len(x)),
      "log_posterior": log_posterior,
      "log_posterior_start_value": np.ones(2 * d)}

for k in range(20):
      np.random.seed(120 + k)
      fkl_parallel_k = []
      bkl_parallel_k = []
      time_parallel_k = []

      full_inds_i = []
      output_i = []
      w_i = []

      for i in coreset_sizes:
            print(k, i)
            fkl_parallel_i = []
            bkl_parallel_i = []
            time_parallel_i = []
            for ind, strat in enumerate([split_randomly, split_based_on_ML]):
                  start = time.time()

                  # Step 1: distribute
                  if ind == 0:
                        full_inds = strat(x)
                  elif ind == 1:
                        gm = GaussianMixture(1)
                        gm.fit(x)
                        params = np.hstack((gm.means_.flatten(), np.diag(gm.covariances_[0])))
                        log_liks = log_likelihood(params, x, None, None)
                        probs = np.abs(log_liks) / np.sum(np.abs(log_liks))
                        probs = probs.flatten()
                        full_inds = distribute(probs)

                  full_inds_i.append(full_inds)

                  # Step 2: run
                  w, output = parallelize(alg = SensitivityBasedFW, x = x, k = int(i // mp.cpu_count()), norm = "2", na = na, distributed_indices = full_inds)

                  time_parallel_i.append(time.time() - start)

                  # Calculate posterior approximation
                  sigma_hat = 1 / (1 + np.sum(w)) * np.identity(d)
                  mu_hat = sigma_hat @ (mu_0 + x.T @ w)

                  fkl_parallel_i.append(gaussian_KL(sigma_full, sigma_hat, mu_full, mu_hat))
                  bkl_parallel_i.append(gaussian_KL(sigma_hat, sigma_full, mu_hat, mu_full))

                  output_i.append(output)
                  w_i.append(w)

            fkl_parallel_k.append(fkl_parallel_i)
            bkl_parallel_k.append(bkl_parallel_i)
            time_parallel_k.append(time_parallel_i)

      fkl_parallel.append(fkl_parallel_k)
      bkl_parallel.append(bkl_parallel_k)
      time_parallel.append(time_parallel_k)

print(f"FKL: {fkl_parallel}")
print(f"BKL: {bkl_parallel}")
print(f"Time: {time_parallel}")

In [ ]:
fkl_parallel = np.array(fkl_parallel)
bkl_parallel = np.array(bkl_parallel)
time_parallel = np.array(time_parallel)

In [ ]:
data = {
    "fkl_sequential": fkl_sequential,
    "bkl_sequential": bkl_sequential,
    "time_sequential": time_sequential,
    "fkl_parallel": fkl_parallel,
    "bkl_parallel": bkl_parallel,
    "time_parallel": time_parallel
}

with open('../data/gaussian.pickle', 'wb') as file:
    pickle.dump(data, file, protocol = pickle.HIGHEST_PROTOCOL)

## Plot

In [ ]:
with open('../data/gaussian.pickle', 'rb') as file:
    data = pickle.load(file)

fkl_sequential = np.array(data['fkl_sequential'])
bkl_sequential = np.array(data['bkl_sequential'])
time_sequential = np.array(data['time_sequential'])
fkl_parallel = np.array(data['fkl_parallel'])
bkl_parallel = np.array(data['bkl_parallel'])
time_parallel = np.array(data['time_parallel'])

coreset_sizes = np.arange(100, 310, 10)

In [ ]:
klsym_sequential = (fkl_sequential + bkl_sequential) / 2
klsym_parallel = (fkl_parallel + bkl_parallel) / 2

In [ ]:
plt.rcParams.update({'font.size': 22})

fig = plt.figure(figsize = (20, 7))

ax12 = fig.add_subplot(121)
ax13 = fig.add_subplot(222)
ax14 = fig.add_subplot(224, sharex = ax13)

ax12.plot(coreset_sizes, np.log(np.median(klsym_sequential, axis = 0)), label = 'Sequential', 
          linestyle = "solid", linewidth = 2, color = 'black')
ax12.plot(coreset_sizes, np.log(np.median(klsym_parallel, axis = 0)[:, 0]), label = 'Random split',
          linestyle = "dashed", linewidth = 2, color = 'dimgray')
ax12.plot(coreset_sizes, np.log(np.median(klsym_parallel, axis = 0)[:, 1]), label = 'ML split',
          linestyle = "solid", marker = "o", linewidth = 2, color = 'maroon')

ax13.spines['bottom'].set_visible(False)
ax13.xaxis.tick_top()
ax13.tick_params(labeltop = False)
ax14.spines['top'].set_visible(False)
ax14.ticklabel_format(useOffset=False)

ax12.set_xlabel("Coreset size")
ax14.set_xlabel("Coreset size")
fig.text(0.07, 0.5, 'Log KL', va = 'center', rotation = 'vertical')
fig.text(0.5, 0.5, 'Seconds', va = 'center', rotation = 'vertical')

d = .015
kwargs = dict(transform = ax13.transAxes, color = 'k', clip_on = False)
ax13.plot((-d, +d), (-d, +d), **kwargs)
ax13.plot((1 - d, 1 + d), (-d, +d), **kwargs)

kwargs.update(transform = ax14.transAxes)
ax14.plot((-d, +d), (1 - d, 1 + d), **kwargs)
ax14.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)

fig.legend()
fig.suptitle('Multivariate Gaussian')

ax13.plot(coreset_sizes, np.median(time_sequential, axis = 0), label = 'Sequential',
          linestyle = "solid", linewidth = 2, color = 'black')
ax14.plot(coreset_sizes, np.median(time_parallel, axis = 0)[:, 0], label = 'Random split',
          linestyle = "dashed", linewidth = 2, color = 'dimgray')
ax14.plot(coreset_sizes, np.median(time_parallel, axis = 0)[:, 1], label = 'ML split',
          linestyle = "solid", marker = "o", linewidth = 2, color = 'maroon')

ax12.grid()
ax13.grid()
ax14.grid()

plt.savefig("../plots/gaussian.eps")
plt.show()

## Distribution

In [ ]:
# Generate data

N = 500
d = 2

np.random.seed(123)

# Generate mean from MVN
theta = np.random.multivariate_normal(mean = np.zeros(d), cov = np.identity(d))

# Generate data
x = np.random.multivariate_normal(mean = theta, cov = np.identity(d), size = N)

# Parameters
sigma_0 = np.identity(d)
sigma = np.identity(d)
mu_0 = np.zeros(d).reshape(-1, 1)

# Full Gaussian posterior
sigma_full = np.linalg.inv(np.linalg.inv(sigma_0) + N * np.linalg.inv(sigma))
mu_full =  sigma_full @ (np.linalg.inv(sigma_0) @ mu_0 + np.linalg.inv(sigma) @ np.sum(x, axis = 0).reshape(-1, 1))

In [ ]:
# Sequential
fkl_sequential = []
bkl_sequential = []
time_sequential = []

na = {"log_likelihood": log_likelihood,
      "log_likelihood_start_value": np.ones(2 * d),
      "S": 100,
      "log_likelihood_gradient": grad_log_likelihood,
      "approx": "MCMC",
      "MCMC_subs_size": int(0.7 * len(x)),
      "log_posterior": log_posterior,
      "log_posterior_start_value": np.ones(2 * d)}

for k in range(1):
      np.random.seed(120 + k)
      fkl_sequential_k = []
      bkl_sequential_k = []
      time_sequential_k = []

      w_seq = []

      for i in [100, 200, 300]:
            print(i, k)
            start = time.time()
            sbfw = SensitivityBasedFW(x)
            w, I = sbfw.run(k = i, likelihood_gram_matrix = None, norm = "2", norm_attributes = na)
            time_sequential_k.append(time.time() - start)

            # Calculate posterior approximation
            sigma_hat = 1 / (1 + np.sum(w)) * np.identity(d)
            mu_hat = sigma_hat @ (mu_0 + x.T @ w)

            fkl_sequential_k.append(gaussian_KL(sigma_full, sigma_hat, mu_full, mu_hat))
            bkl_sequential_k.append(gaussian_KL(sigma_hat, sigma_full, mu_hat, mu_full))

            w_seq.append(w)

      fkl_sequential.append(fkl_sequential_k)
      bkl_sequential.append(bkl_sequential_k)
      time_sequential.append(time_sequential_k)

print(f"FKL: {fkl_sequential}")
print(f"BKL: {bkl_sequential}")
print(f"Time: {time_sequential}")

In [ ]:
# Parallel
fkl_parallel = []
bkl_parallel = []
time_parallel = []

na = {"log_likelihood": log_likelihood,
      "log_likelihood_start_value": np.ones(2 * d),
      "S": 100,
      "log_likelihood_gradient": grad_log_likelihood,
      "approx": "MCMC",
      "MCMC_subs_size": int(0.1 * len(x)),
      "log_posterior": log_posterior,
      "log_posterior_start_value": np.ones(2 * d)}

for k in range(1):
      np.random.seed(120 + k)
      fkl_parallel_k = []
      bkl_parallel_k = []
      time_parallel_k = []

      full_inds_i = []
      output_i = []
      w_i = []

      for i in [100, 200, 300]:
            print(k, i)
            fkl_parallel_i = []
            bkl_parallel_i = []
            time_parallel_i = []
            for ind, strat in enumerate([split_randomly, split_based_on_ML]):
                  start = time.time()

                  # Step 1: distribute
                  if ind == 0:
                        full_inds = strat(x)
                  elif ind == 1:
                        gm = GaussianMixture(1)
                        gm.fit(x)
                        params = np.hstack((gm.means_.flatten(), np.diag(gm.covariances_[0])))
                        log_liks = log_likelihood(params, x, None, None)
                        probs = np.abs(log_liks) / np.sum(np.abs(log_liks))
                        probs = probs.flatten()
                        full_inds = distribute(probs)

                  full_inds_i.append(full_inds)

                  # Step 2: run
                  w, output = parallelize(alg = SensitivityBasedFW, x = x, k = int(i // mp.cpu_count()), norm = "2", na = na, distributed_indices = full_inds)

                  time_parallel_i.append(time.time() - start)

                  # Calculate posterior approximation
                  sigma_hat = 1 / (1 + np.sum(w)) * np.identity(d)
                  mu_hat = sigma_hat @ (mu_0 + x.T @ w)

                  fkl_parallel_i.append(gaussian_KL(sigma_full, sigma_hat, mu_full, mu_hat))
                  bkl_parallel_i.append(gaussian_KL(sigma_hat, sigma_full, mu_hat, mu_full))

                  output_i.append(output)
                  w_i.append(w)

            fkl_parallel_k.append(fkl_parallel_i)
            bkl_parallel_k.append(bkl_parallel_i)
            time_parallel_k.append(time_parallel_i)

      fkl_parallel.append(fkl_parallel_k)
      bkl_parallel.append(bkl_parallel_k)
      time_parallel.append(time_parallel_k)

print(f"FKL: {fkl_parallel}")
print(f"BKL: {bkl_parallel}")
print(f"Time: {time_parallel}")

In [ ]:
plot_coresets(x, w_seq[0], full_inds_i[0], output_i[0], w_i[0], 100, "gaussian")

In [ ]:
plot_coresets(x, w_seq[1], full_inds_i[1], output_i[1], w_i[1], 200, "gaussian")

In [ ]:
plot_coresets(x, w_seq[2], full_inds_i[2], output_i[2], w_i[2], 300, "gaussian")